# More mischievous conventions

(Based on [a question](http://quant.stackexchange.com/questions/12707/) by _Stack Exchange_ user nickos556. Thanks!)

In [1]:
from QuantLib import *
from pandas import DataFrame

#### The case of the two slightly different prices

nickos556 instantiated a fixed-rate bond with semiannual payments and tried to deduce its price from a given yield:

In [2]:
today = Date(27, January, 2011)
Settings.instance().evaluationDate = today

In [3]:
issueDate = Date(28, January, 2011)
maturity = Date(31, August, 2020)
schedule = Schedule(issueDate, maturity, Period(Semiannual),
                    UnitedStates(UnitedStates.GovernmentBond),
                    Unadjusted, Unadjusted,
                    DateGeneration.Backward, False)

bond = FixedRateBond(1, 100.0, schedule,
                     [0.03625],
                     ActualActual(ActualActual.Bond),
                     Unadjusted,
                     100.0)

This can be done either by passing the yield directly...

In [4]:
bond_yield = 0.034921

P1 = bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual)

...or by first setting an engine that uses a corresponding flat term structure.

In [5]:
flat_curve = FlatForward(bond.settlementDate(), bond_yield,
                         ActualActual(ActualActual.Bond),
                         Compounded, Semiannual)

engine = DiscountingBondEngine(YieldTermStructureHandle(flat_curve))
bond.setPricingEngine(engine)
P2 = bond.dirtyPrice()

Surprisingly, the results were different:

In [6]:
DataFrame([(P1,P2)], columns=['with yield', 'with curve'], index=[''])

,with yield,with curve
,101.076816,101.079986


#### What happened?

Mischievous conventions again. The bond uses the Actual/Actual(Bond) convention, which has a requirement: in the case of short or long coupons, we also need to pass a reference start and end date that determine the regular underlying period. Case in point: this bond has a short first coupon.

In [7]:
DataFrame([(as_coupon(c).accrualStartDate(), as_coupon(c).accrualEndDate())
           for c in bond.cashflows()[:-1]],
          columns = ('start date','end date'),
          index = range(1, len(bond.cashflows())))

,start date,end date
1,"January 28th, 2011","February 28th, 2011"
2,"February 28th, 2011","August 31st, 2011"
3,"August 31st, 2011","February 29th, 2012"
4,"February 29th, 2012","August 31st, 2012"
5,"August 31st, 2012","February 28th, 2013"
6,"February 28th, 2013","August 31st, 2013"
7,"August 31st, 2013","February 28th, 2014"
8,"February 28th, 2014","August 31st, 2014"
9,"August 31st, 2014","February 28th, 2015"
10,"February 28th, 2015","August 31st, 2015"


The accrual time for the coupon, starting January 27th 2011 and ending February 28th 2011, must be calculated as:

In [8]:
dayCounter = ActualActual(ActualActual.Bond)

T = dayCounter.yearFraction(Date(28, January, 2011), Date(28, February, 2011),
                            Date(28,August,2010),  Date(28,February,2011))
print(T)

0.08423913043478261


If the coupon were annual, it would be:

In [9]:
print(dayCounter.yearFraction(Date(28, January, 2011), Date(28, February, 2011),
                              Date(28,February,2010),  Date(28,February,2011)))

0.08493150684931507


The corresponding discount factor given the yield is as follows:

In [10]:
y = InterestRate(bond_yield, dayCounter, Compounded, Semiannual)
print(y.discountFactor(T))

0.997087920498809


#### Yield-based calculation

The yield-based calculation uses the above to discount the first coupon, and combines it with discount factors corresponding to the regular coupon times to discount the others. We can write down the full computation:

In [11]:
data = []
for i, c in enumerate(bond.cashflows()[:-1]):
    c = as_coupon(c)
    A = c.amount()
    T = c.accrualPeriod()
    D = y.discountFactor(T)
    D_cumulative = D if i == 0 else D * data[-1][3]
    A_discounted = A*D_cumulative
    data.append((A,T,D,D_cumulative,A_discounted))
data.append((100,'','',D_cumulative,100*D_cumulative))
data = DataFrame(data,
                 columns = ('amount', 'T', 'discount', 'discount (cum.)', 'amount (disc.)'),
                 index = ['']*len(data))
data

,amount,T,discount,discount (cum.),amount (disc.)
,0.305367,0.0842391,0.997088,0.997088,0.304478
,1.812500,0.5,0.982839,0.979977,1.776208
,1.812500,0.5,0.982839,0.963160,1.745727
,1.812500,0.5,0.982839,0.946631,1.715769
,1.812500,0.5,0.982839,0.930386,1.686325
,1.812500,0.5,0.982839,0.914420,1.657386
,1.812500,0.5,0.982839,0.898728,1.628944
,1.812500,0.5,0.982839,0.883305,1.600990
,1.812500,0.5,0.982839,0.868146,1.573515
,1.812500,0.5,0.982839,0.853248,1.546513


The bond price is the sum of the discounted amounts...

In [12]:
print(sum(data['amount (disc.)']))

101.07681646503603


...and, not surprisingly, equals the yield-based bond price.

In [13]:
print(bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual))

101.07681646503603


#### Curve-based calculation

Long story short: the bond engine gets the first discount wrong. Given the curve interface, all it can do is ask for the discounts at the coupon dates, as follows:

In [14]:
data = []
for c in bond.cashflows()[:-1]:
    A = c.amount()
    D = flat_curve.discount(c.date())
    A_discounted = A*D
    data.append((A,D,A_discounted))
data.append((100.0,D,100.0*D))
data = DataFrame(data,
                 columns = ('amount', 'discount', 'amount (disc.)'),
                 index = ['']*len(data))
data

,amount,discount,amount (disc.)
,0.305367,0.997119,0.304487
,1.812500,0.980008,1.776264
,1.812500,0.963190,1.745782
,1.812500,0.946661,1.715823
,1.812500,0.930415,1.686378
,1.812500,0.914449,1.657438
,1.812500,0.898756,1.628995
,1.812500,0.883332,1.601040
,1.812500,0.868174,1.573565
,1.812500,0.853275,1.546561


The result equals the curve-based price.

In [15]:
print(sum(data['amount (disc.)']))

101.0799861183387


In [16]:
print(bond.dirtyPrice())

101.0799861183387


The problem is that the first call to the `discount` method, that is,

In [17]:
flat_curve.discount(Date(28, February, 2011))

0.9971191880350325

results in a call to:

In [18]:
print(dayCounter.yearFraction(Date(28, January, 2011), Date(28, February, 2011)))

0.08333333333333333


Compare this to the correct one:

In [19]:
T = dayCounter.yearFraction(Date(28, January, 2011), Date(28, February, 2011),
                            Date(28,August,2010),  Date(28,February,2011))
print(T)

0.08423913043478261


Does this account for the difference in price? Yes, it does. The two prices can be reconciled as follows:

In [20]:
P_y = bond.dirtyPrice(bond_yield, bond.dayCounter(), Compounded, Semiannual)
D_y = y.discountFactor(T)

P_c = bond.dirtyPrice()
D_c = flat_curve.discount(Date(28, February, 2011))

print(P_y)
print(P_c*(D_y/D_c))

101.07681646503603
101.0768164650361
